In [44]:
class pdfToText:
    def __init__(self,fileName):
        self.fileName=fileName
        #holds regrexed text
        self.text=""
        #holds string of pdf data directly
        self.rawtext=""
        self.refinedText=""
        self.refinedWordWholeList=[]
        self.lineList=[]
        self.wordListEachLine=[]
        self.totalWordList=[]
        self.tagList=[]
    def convertPDFtoText(self):
        import PyPDF2
        pdfFileObj = open(self.fileName,'rb')     #'rb' for read binary mode
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        noOfPages=pdfReader.numPages
        for pages in range(0,pdfReader.numPages):
            pageObj = pdfReader.getPage(pages)          #'9' is the page number
            self.rawtext+=pageObj.extractText()
        self.normalizeText()
        self.sizeOfText()
    def normalizeText(self):
        import nltk
        self.removeSpecialrefinedTextCharacterFromString()
        self.tokenizeToLine()
        self.tokenizeToWords()
        self.tokenizeToWordsEachLine()
        stopwords=nltk.corpus.stopwords.words('english')
        self.refinedWordWholeList=[word for word in self.totalWordList if word not in stopwords]
        wnl = nltk.WordNetLemmatizer()
        self.refinedWordWholeList=[wnl.lemmatize(word) for word in self.refinedWordWholeList]
        self.refinedText=' '.join(self.refinedWordWholeList)
    def removeSpecialrefinedTextCharacterFromString(self):
        import re
        self.text+= re.sub(r'[^a-zA-Z ]',r'',self.rawtext)
        
    def printAllVariables(self):
        print("\nself.text:\n"+self.text)
        print("\nself.rawtext:\n"+self.rawtext)
        print("\nself.lineList:\n")
        for line in self.lineList:
            print(line)
        print("\nself.wordListEachLine:\n")
        for line in self.wordListEachLine:
            print(line)
        print("\nself.tagList:\n")
        for line in self.tagList:
            print(line)
        print("\nself.refinedWordWholeList:\n")
        print(self.refinedWordWholeList)
        print("\nself.totalWordList:\n")
        print(self.totalWordList)
       
      
    def sizeOfText(self):
        return len(self.text)
    
    def tokenizeToLine(self):
        import nltk
        self.lineList=nltk.sent_tokenize(self.rawtext)
        
    def tokenizeToWordsEachLine(self):
        import nltk
        for line in self.lineList:
            self.wordListEachLine.append(nltk.word_tokenize(line))
            
    def tokenizeToWords(self):
        import nltk
        self.totalWordList=nltk.word_tokenize(self.text)
            
    def textVsWordGraph(self,listName=None):
        import nltk
        if listName is None:
            listName=self.totalWordList
        FDIST=nltk.FreqDist(listName)
        FDIST.plot(10,linewidth=1.0)
        print("comparing top 20 word occurance in whole text")
        
   
    def categorizePOSTag(self,listName=None):
        import nltk
        if listName is None:
            listName=self.totalWordList
        self.tagList=nltk.pos_tag(listName)
        

In [45]:
class databaseHandler:
    def __init__(self,password="ST.xavier51",databaseName="cvRanking"):
        import pymysql
        try:
            self.database=pymysql.connect("localhost","root","ST.xavier51","cvRanking")
            self.cursor = self.database.cursor()
        except Exception as e:
            print("error info %s"%e)
    def entryTheCV(self,data_CV):
        try:
            dataReturned=self.checkDuplication(data_CV[1])
            if(len(dataReturned)==0):
                add_CV="""insert into recordentry1
                    (checkRecord,recordname)
                    values(%s,%s)"""
    #             add_CV="""insert into recordentry
    #                 (idrecordEntry,checkRecord,recordname)
    #                 values(%s,%s,%s)"""
                #data_CV=("yes","ram")
                print(data_CV)
                self.cursor.execute(add_CV,data_CV)
                self.database.commit()
                print("CV is entered")
                return True;
            else:
                print("CV already present\n location at %s"%str(dataReturned))
                return False
        except Exception as e:
            print("in entryTheCV\t"+str(e))
            return False
    def checkDuplication(self,name_CV):
        try:
            check_CV="""select* from recordentry as re where re.recordname=%s"""
            self.cursor.execute(check_CV,name_CV)
            result = self.cursor.fetchall()
        except Exception as e:
            print("in process checkDuplication\t"+str(e))
        return result
            

        
    def closeConnection(self):
        self.database.close()


In [46]:
class CV_manager:
    def __init__(self):
        self.database=databaseHandler()
        from sklearn.feature_extraction.text import CountVectorizer
        self.vectorizer=CountVectorizer()
        self.CVList=[]
        #refined word coll
        self.textCollectionForDM=[]
        self.wordCollection=[]
        self.tagCollection=[]
        self.documentMatrix=[]
        self.featureVectorDict={}
        self.normalizedVectorValue=[]
        self.fileNamesWithPath=[]

    def list_CVs(self,rootPath):
        import os
        for root, dirs, files in os.walk(rootPath):
            for name in files:
                self.fileNamesWithPath.append(os.path.join(root, name))
    def addCVFromSavedList(self):
        for file in self.fileNamesWithPath:
            if(self.database.entryTheCV(("yes",file))):
                pdf=pdfToText(file)
                self.CVList.append(pdf)
    def doConversion(self):
        for cv in self.CVList:
            cv.convertPDFtoText();
    def size(self):
        return len(self.CVList)
    def compareTotalLines(self):
        import matplotlib.pyplot as plt
        for cvNum,cv in enumerate(self.CVList):
            if not cv.text:
                #conversion not done
                cv.convertPDFtoText()
            #conversion handles all the tokenization tasks
            plt.plot(cvNum+1,len(cv.lineList),'g^',linewidth=2.0)
            plt.xlabel('CV')
            plt.ylabel('totalLines')
        plt.show()
    def compareTotalWords(self):
        import matplotlib.pyplot as plt
        for cvNum,cv in enumerate(self.CVList):
            if not cv.text:
                #conversion not done
                cv.convertPDFtoText()
            #conversion handles all the tokenization tasks
            plt.plot(cvNum+1,len(cv.refinedWordWholeList),'g^',linewidth=2.0)
            plt.xlabel('CV')
            plt.ylabel('totalLines')
        plt.show()
    def PopualarWords(self,number=20):
        import nltk
        for cvNum,cv in enumerate(self.CVList):
            if not cv.text:
                #conversion not done
                cv.convertPDFtoText()
            FDIST=nltk.FreqDist(cv.refinedWordWholeList)
            FDIST.plot(number,linewidth=1.0)
            print("comparing top %d word occurance in whole text of %d"%(number,cvNum+1))
    def collectAllWordsFromCVList(self):
        for cv in self.CVList:
            for word in cv.refinedWordWholeList:
                self.wordCollection.append(word)
    def frequentWordsGraph(self,wordList=None,number=20):
        import nltk
        if wordList is None:
            wordList=self.wordCollection
        FDIST=nltk.FreqDist(wordList)
        FDIST.plot(number,linewidth=1.0)
        print("comparing top %d word occurance in whole CV collection"%(number))
    def tagAllWordsFromCVList(self):
        import nltk
        if self.wordCollection is None:
            self.collectAllWordsFromCVList()
        self.tagCollection=(nltk.pos_tag(self.wordCollection))
    def getRequierdPOSGraph(self,selectionList):
        import nltk
        NNPList=[]
        if self.tagCollection is None:
            self.tagAllWordsFromCVList()
        for word in self.tagCollection:
            if word[-1] in selectionList:
                NNPList.append(word)
        self.frequentWordsGraph(NNPList)
    def convertToDocumentMatrix(self,frequency):
        for cv in self.CVList:
            self.textCollectionForDM.append(cv.refinedText)
        self.documentMatrix=self.vectorizer.fit_transform(self.textCollectionForDM)
        print (self.vectorizer.vocabulary_)
        #print(self.vectorizer.vocabulary_)
        #self.getFrequencyTableOfWords(frequency)
        #self.normalizeFeatureVectors()
            #to comment or uncomment ctrl+/
    def getFrequencyTableOfWords(self,frequency):
        import numpy as np
        import json
        import matplotlib.pyplot as plt
        vocab=self.vectorizer.get_feature_names()
        dist = np.sum(self.documentMatrix.toarray(), axis=0)
        for tag, countTF in zip(vocab, dist):
            if(countTF > frequency):
                self.featureVectorDict.update({tag:int(countTF)})
        print(self.featureVectorDict)
        with open("C:\\Users\\sagar\\Desktop\\CV coll\\documentMatrixWord.txt","w") as file:
            json.dump(self.featureVectorDict, file)
        file.close()
    def normalizeFeatureVectors(self):
        from sklearn.preprocessing import normalize
        import numpy as np
        vectorValue=[]
        for key in self.featureVectorDict:
            vectorValue.append(self.featureVectorDict[key])
        datas = np.asarray(vectorValue, dtype=np.float)
        datas =normalize(datas, norm='l2')
        #converting numpy.array to list
        for r in datas:
            for column in r:
                self.normalizedVectorValue.append(column)
        print(self.normalizedVectorValue)
                
        
        


In [47]:
cmp=CV_manager()
cmp.list_CVs("C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director")
cmp.addCVFromSavedList()
cmp.doConversion()


('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Agnes-Szelag.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Alison-Haller.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Brad-Blackburn.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Bruce-Jones.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Carolyn-Carmel.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Christina-Le-Sesne.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Christopher-Mackey.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Cosmo-Nardozza.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director\\Craig-Thacker.pdf')
CV is entered
('yes', 'C:\\Users\\sagar\\

In [48]:
 cmp.convertToDocumentMatrix(30)

{'agnes': 140, 'szelagcreative': 4741, 'directoroakland': 1548, 'caemail': 694, 'indeed': 2429, 'indeedcomragnesszelagedcffaaauthorized': 2430, 'work': 5282, 'us': 5076, 'employerwork': 1746, 'experiencecreative': 1876, 'directorberkeley': 1494, 'resource': 4218, 'center': 774, 'online': 3458, 'education': 1691, 'uc': 5021, 'berkeley': 539, 'direct': 1470, 'medium': 3096, 'art': 344, 'production': 3906, 'concept': 997, 'preproduction': 3800, 'storyboards': 4633, 'studio': 4674, 'onlocation': 3462, 'productiongraphic': 3910, 'design': 1305, 'motion': 3239, 'graphic': 2175, 'postproduction': 3761, 'distribution': 1602, 'with': 5264, 'management': 2938, 'team': 4783, 'create': 1151, 'manage': 2933, 'vision': 5164, 'culture': 1197, 'mission': 3184, 'statementoperational': 4608, 'structure': 4665, 'workflow': 5290, 'consisting': 1028, 'instructionaldesigners': 2566, 'product': 3903, 'manager': 2952, 'lead': 2771, 'editor': 1678, 'camera': 709, 'people': 3622, 'artist': 351, 'animatorsand': 

In [ ]:
# class databaseHandler:
#     def __init__(self,password="ST.xavier51",databaseName="cvRanking"):
#         import pymysql
#         try:
#             self.database=pymysql.connect("localhost","root","ST.xavier51","cvRanking")
#             self.cursor = self.database.cursor()
#         except Exception as e:
#             print("error info %s"%e)
#     def entryTheCV(self,data_CV):
#         try:
# #             dataReturned=self.checkDuplication(data_CV[1])
# #             if(len(dataReturned)==0):
#                 add_CV="""insert into recordentry
#                     (checkRecord,recordname)
#                     values(%s,%s)"""
#     #             add_CV="""insert into recordentry
#     #                 (idrecordEntry,checkRecord,recordname)
#     #                 values(%s,%s,%s)"""
#                 #data_CV=("yes","ram")
#                 print(data_CV)
#                 self.cursor.execute(add_CV,data_CV)
#                 self.database.commit()
#                 print("CV name is entered in database")
#         except Exception as e:
#             print("in entryTheCV\t"+str(e))
#     def checkDuplication(self,name_CV):
#         try:
#             check_CV="""select* from recordentry as re where re.recordname=%s"""
#             self.cursor.execute(check_CV,name_CV)
#             result = self.cursor.fetchall()
#         except Exception as e:
#             print("in process checkDuplication\t"+str(e))
#         if(len(result)==0):  
#             return False
#         else:
#             return True  
#     def closeConnection(self):
#         self.database.close()

In [49]:
# cmp.compareTotalLines()


In [50]:
# cmp.compareTotalWords()

In [2]:
textCollectionForDM=['i am sagar sagar','i is hero sagar','i ass sagar']
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
documentMatrix=[]
vectorizer=CountVectorizer(stop_words='english',min_df=1)
documentMatrix=vectorizer.fit_transform(textCollectionForDM)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(documentMatrix)
documentMatrixNormalized=tfidf.transform(documentMatrix)
print(documentMatrix.todense())
print (vectorizer.vocabulary_)
print(type(documentMatrix))

[[0 0 2]
 [0 1 1]
 [1 0 1]]
{'sagar': 2, 'hero': 1, 'ass': 0}
<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
from scipy import sparse
with open("C:\\Users\\sagar\\Desktop\\CV coll\\documentMatrixsaas.txt","w") as file:
#Save
sparse.save_npz('data_sparse.npz', file)

#Load
data = sparse.load_npz("data_sparse.npz")

In [52]:

# import numpy as np
# # print(documentMatrix.toarray())
# # print(documentMatrixNormalized.toarray())
# # x=[1,2,3,4]
# # y=[5,6,7,8]
# # vocab=vectorizer.get_feature_names()
# # dist = documentMatrix.toarray()
# print(type(documentMatrix))
# # for vectors in dist:
# #     print(vectors)
# from sklearn.preprocessing import normalize
# data = np.array(documentMatrix.toarray())
# print(data)
# data =normalize(data, norm='l2')
# print(data)

# # for item,freqn in zip(vocab, dist):
# #         if(freqn>2):
# #             print(item,freqn)
# #print(count)


In [53]:

# import numpy as np
# import matplotlib.pyplot as plt
# vocab=vectorizer.get_feature_names()
# dist = np.sum(documentMatrix.toarray(), axis=0)
# print(dist)
# yLocation=np.arange(len(dist))
# width = 0.35
# fig, ax = plt.subplots()
# for tag, count in zip(vocab, dist):
#     print(count, tag) 
# plt.bar(yLocation, dist, align='center', alpha=0.5)
# plt.xticks(yLocation, vocab)
# plt.ylabel('words')
# plt.title('comp')
 
# print(plt.show())

In [54]:
# print(vectorizer.vocabulary_)
# for x in documentMatrix:
#     print(x[0][0])

In [55]:
# import numpy as np
# from sklearn.preprocessing import normalize
# vectorValue=[1,2,3,4,5]
# data = np.asarray(vectorValue,dtype=np.float)
# data
# data1 =normalize(data, norm='l2')
# x=[]
# for r in data1:
#     print(data1[0])
#     for column in r:
#         print(column)
#         x.append(column)
#         #print(data1[0][column])
# x

In [56]:
#len(cmp.normalizedVectorValue)

In [57]:
# import numpy as np
# from sklearn.cluster import MeanShift
# from sklearn.datasets.samples_generator import make_blobs
# import matplotlib.pyplot as plt
# from matplotlib import style
# style.use("ggplot")
# #plt.scatter(cmp.normalizedVectorValue)
# #plt.show()
# ms = MeanShift()
# ms.fit(cmp.normalizedVectorValue)
# labels = ms.labels_
# cluster_centers = ms.cluster_centers_
# print(cluster_centers)

In [58]:
# filepath=("C:\\Users\\sagar\\Desktop\\CV coll")
# import os
# try:
#     if(os.path.exists(filepath)):
#         for filename in os.listdir(filepath):
#             print(filename)
#             print(os.path.isfile(filename))
#     else:
#         print("path not found")
#             #print("directory")
# #         if filename.endswith(".pdf"):
# #                 print(filename)
# #         else:
# #             print("files not found in required format")
# except:
#     print("location not valid")
# #pdfFileObj = open(filepath+'.pdf','rb')

In [59]:
# filepath=("C:\\Users\\sagar\\Desktop\\CV coll\\kailesh")
# import os

# def list_files(dir):
#     r = []
#     for root, dirs, files in os.walk(dir):
#         for name in files:
#             print(name)
#             r.append(os.path.join(root, name))
#     return r
# list_files(filepath)

In [79]:
words=['eat','eats','eating']
from nltk.stem import WordNetLemmatizer
x=''
wnl = nltk.WordNetLemmatizer()
for word in words:
    x=wnl.lemmatize("eats")
    print(x)
#refinedText=' '.join(self.refinedWordWholeList)

eats
eats
eats


In [67]:
refinedWordWholeList

['eat', 'eats', 'eating']

In [85]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize('happier')


'happier'

In [86]:
from nltk.stem import PorterStemmer

In [87]:
stemmer = PorterStemmer()

In [88]:
singles = [stemmer.stem(plural) for plural in words]

In [93]:
print(singles)
print(list(set(singles)))

['eat', 'eat', 'eat']
['eat']


In [90]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))

cat
cactus
goose
rock
python
good
best
run
run


In [106]:
#words=['eat','eats','eating','happy','happying','cry','crying']
words=['i am eating apple','i am eat orange']
word=nltk.word_tokenize(words)
print(word)
def findDocumentMatrix(totalCVText):
        import nltk
        from sklearn.feature_extraction.text import CountVectorizer
        textList=[]
        word=nltk.word_tokenizer(words)
        print(word)
        textList=lemmatize(totalCVText)
        textList=Stemmer(totalCVText)
        vectorizer=CountVectorizer(stop_words='english')
        documentMatrix=vectorizer.fit_transform(textList)
        vocabulary=vectorizer.vocabulary_
        print(documentMatrix)
        print(vocabulary)
        return documentMatrix,vocabulary
def Stemmer(totalCVText):
        from nltk.stem import PorterStemmer
        stemmer = PorterStemmer()
        singles = [stemmer.stem(plural) for plural in totalCVText]
        print(singles)
        singles=list(set(singles))
        print(singles)
        return singles
def lemmatize(totalCVText):
        from nltk.stem import WordNetLemmatizer
        stemmer = WordNetLemmatizer()
        singles = [stemmer.lemmatize(plural) for plural in totalCVText]
        print(singles)
        singles=list(set(singles))
        print(singles)
        return singles
findDocumentMatrix(words)

AttributeError: module 'nltk' has no attribute 'word_tokenizer'

In [115]:
#words='i am eating apple i am eat orange'
words=['i am eating apple','i am eat orange']
wordList=[]
for word in words:
    word=nltk.word_tokenize(word)
    wordList.append(word)
print(wordList)

[['i', 'am', 'eating', 'apple'], ['i', 'am', 'eat', 'orange']]


In [111]:
words=['i am eating apple','i am eat orange']
text=' '.join(words)

In [112]:
text

'i am eating apple i am eat orange'

In [138]:
import json
CVTextColl=["hari","ram"]
with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVTextsss.json","w") as file:
    json.dump(CVTextColl,file)
file.close()
CVTextColl=[]
with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVTextsss.json","r") as file:
    CVTextColl=json.load(file)
file.close()


In [140]:
CVTextColl[0]

'hari'